# Use Google Drive

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
% cd / content / gdrive / MyDrive / Colab Notebooks/DL
!ls


Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/DL
1.png		       Data.csv		 references  train_id.csv
2021VRDL_HW1_datasets  Info.json	 test	     valid_id.csv
bg.jpg		       Object-Detection  train


# Use GPU

In [2]:
# Show the GPU info

!nvidia-smi


Wed Nov 24 13:11:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
print(torch.__version__)
torch.cuda.is_available()


1.10.0+cu111


True

# Git clone your project

In [4]:
!git clone https://github.com/YCHung1998/Object-Detection.git

fatal: destination path 'Object-Detection' already exists and is not an empty directory.


In [5]:
%cd Object-Detection

/content/gdrive/MyDrive/Colab Notebooks/DL/Object-Detection


# Install Dependencies

In [6]:
%cd yolov5
%pip install -qr requirements.txt  # install
%cd /content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection

/content/gdrive/MyDrive/Colab Notebooks/DL/Object-Detection/yolov5
     |████████████████████████████████| 596 kB 5.1 MB/s 
/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection


# Wget testing data 

In [7]:
# You must import the below 5 packages
import os
import cv2
import time
from tqdm import tqdm
from google_drive_downloader import GoogleDriveDownloader as gdd


In [1]:
# 1. Download the testing data
file_id = '1Fm-avdeNgzhPxhvia0iw9yZzcoOggy7I'
gdd.download_file_from_google_drive(file_id=file_id,
                                    dest_path='./test.zip',
                                    unzip=True)
# 2. Read the image_name and put them into the list
# You need to modify the path to fit your test_folder

data_listdir = os.listdir("test/")


NameError: name 'gdd' is not defined

In [9]:
len(data_listdir)

13068

# load model

In [10]:
%cd / content / gdrive / My Drive / Colab Notebooks / DL / Object-Detection / yolov5


/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/yolov5


In [11]:
from utils.datasets import LoadStreams, LoadImages
from utils.torch_utils import select_device
from pathlib import Path
import json
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random
from models.experimental import attempt_load
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords,
    xyxy2xywh, strip_optimizer, set_logging)
from utils.torch_utils import select_device
from models.experimental import attempt_load


In [12]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = "cpu"
imgsz = 640

weights = 'runs/train/exp/weights/best.pt'
model = attempt_load(weights, map_location=device)  # load FP32 model
imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size


Fusing layers... 
Model Summary: 290 layers, 20889303 parameters, 0 gradients, 48.1 GFLOPs


# Run inferene and bench mark

In [13]:
%cd /content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection

/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection


In [14]:
pwd

'/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection'

In [21]:
# Test your inference time
TEST_IMAGE_NUMBER = 100  # This number is fixed.
test_img_list = []

# Read image (Be careful with the image order)
data_listdir.sort(key=lambda x: int(x[:-4]))
path = r"/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test"
for img_name in data_listdir[:TEST_IMAGE_NUMBER]:
    img_path = os.path.join(path, img_name)
    test_img_list.append(img_path)
print(len(test_img_list))
print(test_img_list[:10])

start_time = time.time()
model.eval()
JSON = []
for img_path in tqdm(test_img_list):
    dataset = LoadImages(img_path, img_size=imgsz)
    for path, img, im0, vid_cap, s in dataset:
        # get image id
        img_id = path.split('\\')[-1]  # '100000.png'

        # predict
        img = torch.from_numpy(img).float().to(device)
        img /= 255.0
        img = img.unsqueeze(0)

        # pred = model(img, augment=True)[0]

        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred)

        for det in pred:  # for prediction in mini-batch size

            if len(det):
                # (x, y, x, y, score, cls_id)
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape)

                for *xyxy, conf, cls in det:
                    '''
                    {'image_id': 100000, 'category_id': cls_id, 'score': score,
                     'bbox': [x, y, w, h]}
                    '''
                    xyxy = [float(c) for c in xyxy]
                    conf = float(conf)
                    cls = int(cls)

                    x1, y1, x2, y2 = xyxy
                    w = x2 - x1
                    h = y2 - y1
                    bbox = {'image_id': img_id,
                            'category_id': cls,
                            'score': conf,
                            'bbox': [x1, y1, w, h]}
                    JSON.append(bbox)
end_time = time.time()
print("\nInference time per image: ",
      (end_time - start_time) / len(test_img_list))
# Remember to screenshot!


100
['/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/117.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/162.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/196.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/203.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/256.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/319.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/345.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/372.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/397.png', '/content/gdrive/My Drive/Colab Notebooks/DL/Object-Detection/test/401.png']


100%|██████████| 100/100 [00:05<00:00, 19.61it/s]


Inference time per image:  0.051053750514984134


# Generate answer.json for submission on Codalab

- Please use the inference.py to generate the answer.json which will be save in the testing folder
- Warning : Please insure that the columns 27 in inference.py  
as the root represnted as "/DL_HW2/YC/datasets/Testing/images/100000.png"

```
img_id = path.split('/')[-1] # '100000.png' 
```
as the root represnted as "\\DL_HW2\\YC\\datasets\\Testing\\images\\100000.png"

```
img_id = path.split('\\')[-1] # '100000.png' 
```